In [1]:
print("ok")

ok


In [2]:
import unstructured
from importlib.metadata import version

In [3]:
print(version("unstructured"))

0.18.15


In [4]:
import os
from dotenv import load_dotenv

if os.getenv('AZURE_OPENAI_API_KEY'):
    print("AzureOpenAI API key loaded successfully")
else:
    print("API KEY not found")

AzureOpenAI API key loaded successfully


In [6]:
from unstructured.partition.auto import partition
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.html import partition_html
from unstructured.partition.md import partition_md
from unstructured.partition.docx import partition_docx
from unstructured.partition.xlsx import partition_xlsx
from unstructured.partition.pptx import partition_pptx
from unstructured.partition.image import partition_image

from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements

from unstructured.documents.elements import (
    Title,
    NarrativeText,
    Table,
    ListItem,
    Image,
    Header,
    Footer,
    Text,
    ElementMetadata
)

import json
from pathlib import Path

f:\sourab\data_parsing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
html_path = 'sample_documents\sample.html'
elements = partition(html_path)

for i, element in enumerate(elements,1):
    element_type = type(element).__name__
    text_preview = element.text[:100]+"..." if len(element.text)> 100 else element.text
    print(f"[{i}] {element_type}: {text_preview}")
    print("-"*60)


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


[1] Title: 1. Overview
------------------------------------------------------------
[2] NarrativeText: Document parsing is the process of analyzing and extracting structured information from various docu...
------------------------------------------------------------
[3] Title: 1.1 Key Benefits
------------------------------------------------------------
[4] ListItem: Automated data extraction
------------------------------------------------------------
[5] ListItem: Structured content analysis
------------------------------------------------------------
[6] ListItem: Integration with AI/ML pipelines
------------------------------------------------------------
[7] ListItem: Support for multiple formats
------------------------------------------------------------
[8] Title: 2. Core Features
------------------------------------------------------------
[9] NarrativeText: Modern document parsers offer a variety of features:
------------------------------------------------------------
[10] 

In [9]:
title_element = [e for e in elements if isinstance(e, Title)]

if title_element:
    first_title = title_element[0]

    print("Element Text: ")
    print(first_title.text)
    print('-'*60)

    print("Element Metadata: ")
    meta_data = first_title.metadata

    print(meta_data)
    meta_data_dict = meta_data.to_dict()
    for key, value in meta_data_dict.items():
        if value is not None:
            print(f"   {key}: {value}")

Element Text: 
1. Overview
------------------------------------------------------------
Element Metadata: 
   category_depth: 1
   last_modified: 2025-12-30T05:14:11
   languages: ['eng']
   file_directory: sample_documents
   filename: sample.html
   filetype: text/html


In [10]:
def analyze_elements(elements):
    type_count = {}
    for element in elements:
        element_type = type(element).__name__
        type_count[element_type] = type_count.get(element_type,0) + 1
    print("Element Distribution: ")
    print("-"*30)
    for elem_type, count in sorted(type_count.items()):
        print(f"  {elem_type}: {count}")

In [11]:
type_count = analyze_elements(elements)

Element Distribution: 
------------------------------
  ListItem: 8
  NarrativeText: 6
  Table: 1
  Title: 5


In [13]:
narrative_texts = [e for e in elements if isinstance(e, NarrativeText)]
print(f"Found {len(narrative_texts)} Narrative elements")
for i, text in enumerate(narrative_texts[:5],1):
    print(f"{i}. {text.text}\n")

Found 6 Narrative elements
1. Document parsing is the process of analyzing and extracting structured information from various document formats. This includes PDFs, Word documents, HTML pages, and more.

2. Modern document parsers offer a variety of features:

3. Retrieval-Augmented Generation (RAG) systems benefit significantly from proper document parsing:

4. "Effective document parsing is the foundation of any successful RAG implementation."

5. Here's a simple example of using a document parser:



In [14]:
table_elements = [e for e in elements if isinstance(e,Table)]
print(f"Found {len(table_elements)} Table elements: \n")
for i, table in enumerate(table_elements):
    print(f"Table {i+1}: \n")
    print(table.text)
    print()

    if hasattr(table.metadata, 'text_as_html') and table.metadata.text_as_html:
        print("HTML representation available in metadata.text_as_html")
    print("-"*60)

Found 1 Table elements: 

Table 1: 

Feature Description Use Case OCR Support Optical Character Recognition for scanned documents Scanned PDFs, Images Table Extraction Structured table data extraction Financial reports, Data tables Layout Analysis Understanding document structure Academic papers, Legal documents Image Processing Extract and classify images Technical manuals, Presentations

HTML representation available in metadata.text_as_html
------------------------------------------------------------


### **Partitioning Strategies**

In [15]:
import urllib.request
import os

os.makedirs("sample_documents", exist_ok=True)
pdf_url = "https://arxiv.org/pdf/2408.09869"
pdf_path = "sample_documents\docling_paper.pdf"

if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}")
    urllib.request.urlretrieve(pdf_url)
    print(f"Downloaded to {pdf_path}")
else:
    print(f"PDF exist at: {pdf_path}")

PDF exist at: sample_documents\docling_paper.pdf


In [ ]:
import time
print("Strategy: AUTO (default)")
print("="*60)

start_time = time.time()
